In [1]:
import requests
import time
from bs4 import BeautifulSoup
import re
from urllib import parse 
import time
from IPython.display import Image
import sqlite3
import sys

In [136]:
headers = {"accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3","user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

## 순서
https://kin.naver.com/people/expert/index.nhn?type=DOCTOR&edirId=44 <br>
`이처럼 edirId= 44 ~ 48 까지가 치과 종목은 다 다름` <br>
`페이지 옮기면 &page=2 가 붙음`

{44: "치아교정 전문가", 45: }

1. 처음 edirid = 44에 들어감
2. 거기서 list_summary 파싱하면 어떤 전문가 인지 나옴
3. pro_list 태그 파싱하면 밑에 li의 개수로 전체 리스트 가져올 수 있음.
4. div태그의 a태그 href을 가져오면 전문가의 페이지로 이동할 수 있음.
5. .profile_section2에서 이름 : pro_name  /  특정 병원 위치 : position / 세부분야 pro_info dd 태그
6. #au_board_list 태그 파싱하면 그 페이지꺼 다 가져올 수 있음
7. 글 파싱할때는 제목 : #content .title  / 질문내용 .c-heading__content / .answer-content__list의 div들을 가져옴
    , 한개씩들어간후 .title들어가서 a태그의 text 값에 의사 이름이 있으면 True하고 거기에 있는 ._endContentsText c-heading-answer__content-user 파싱
8. page를 1씩 증가해 가면서 파싱
9. 만약 페이지를 증가해가다 없으면 2번으로 들어가 다음 전문가 탐색함.


## 지식인 게시글 문제
1. https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=331221984
2. https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332118173

폼이 다름

## 데이터 베이스
#### 지식인
1. 전문분야 - 의치, 임플란트/ 잇몸질환/ 충치, 치아질환/ 치야교정/ 치아 유지, 관리
2. 의사 info - 이름, 세부분야, 치과이름,  
    + 세부분야 - 없을 수 도 있음. 그럴 경우 None
    + 치과 이름 없을 수도 있음.
3. 게시글 - 질문 제목, 질문 내용, 상담 내용


In [204]:
import pymysql
 
# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='root', password='tjwndud',
                       db='bigdatabio07', charset='utf8')
 
# Connection 으로부터 Cursor 생성
cur = conn.cursor()

In [220]:
def download(method, url, headers=None, cookies=None, param = None, data = None, timeout=1, maxretries = 3):
    try:
        resq = requests.request(method, url,params = param, data=data, headers = headers, cookies = cookies)
        resq.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if (500<= e.response.status_code < 600) and maxretries>0:
            time.sleep(timeout)
            print("재시도")
            download(method, url,timeout=timeout,maxretries=maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resq

In [206]:
start_url = "https://kin.naver.com/people/expert/index.nhn"
params = {"edirId" : 44, "type": "DOCTOR", "page":1}
html = download("get", start_url,headers = headers, param = params)

In [249]:
start_url = "https://kin.naver.com/people/expert/index.nhn"
params = {"edirId" : 44, "type": "DOCTOR", "page":1}
global_num = 0
while True:
    html = download("get", start_url,headers = headers, param = params)
    dom = BeautifulSoup(html.text, "lxml")

    doc_list = [x.select("a")[0]['href'] for x in dom.select(".pro_list li")]
    
    if(doc_list == []):
        print("=============끝==============")
        break
    
    DocThe_page = [parse.urljoin(start_url, x).replace('index','answerList') for x in doc_list]
    
    goPage(DocThe_page)
    
    params['page']+=1

20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332325044
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332324376
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332313442
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332310636
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332308337
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332297888
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70110&docId=332244628
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=332234663
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=332269535
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70110&docId=332260662
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=332258265
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70110&docId=332255451
----------
h

https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=330738972
----------
20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=330834152
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=330834038
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=330833907
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=330825691
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=330790610
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=330885627
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=330802244
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=330800006
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=330782562
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=330779569
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=330777736
----------
h

https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=329093650
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=329086289
----------
20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=329082527
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=329061399
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=329071348
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=329064972
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=329051072
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=328934653
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=328988351
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=328849182
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=329021342
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=328935063
----------
h

https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70111&docId=327634990
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=327676599
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70203&docId=327675044
----------
20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=327675948
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=327605758
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70110&docId=327604687
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=327603754
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=327578923
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=327606838
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=327535441
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=327543998
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=327531585
----------
h

https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=325951543
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=326000711
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=325885024
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=325998724
----------
20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=325997292
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=325991406
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=325988370
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=325970184
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=325940372
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=325923191
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=325923918
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=325942318
----------
h

https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=324729108
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=324722535
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=324716636
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=324696140
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=324683395
----------
20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=324619557
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=324597123
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=324590105
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=324436926
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=324592774
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=324194772
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=324141136
----------
h

https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323516298
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=323575912
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323365024
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323454792
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323405980
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323470225
----------
20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323503062
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323386512
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=323477719
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323394971
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=323465818
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=323477992
----------
h

https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=322142336
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=322147683
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=322109127
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=322083057
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=322082815
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=322077328
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70202&docId=322006197
----------
20
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=322001901
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70201&docId=321995648
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=321991897
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=321983043
----------
https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70205&docId=321981510
----------
h

TypeError: 'NoneType' object is not subscriptable

In [238]:
def goPage(pageList):
    for i in pageList:
        html = download("get", i, headers = headers)
        dom = BeautifulSoup(html.text, "lxml")
        pro_info = dom.select_one(".pro_intro")
        pro_name = pro_info.select_one("dt").text
        pro_Loc = pro_info.select_one(".position").text
        curNum=1
        pro_dict = {"name":pro_name, "loc": pro_Loc}
        pushDb(pro_dict,0)
        while True:
            html = download("get", i, headers = headers, param = {"page":curNum})
            #print(html.url)
            dom = BeautifulSoup(html.text, "lxml")
            #print(dom)
            boardList = [x.select_one(".title a")['href'] for x in dom.select("#au_board_list tr")]
            print(len(boardList))
            if(len(boardList) == 0):
                break
            boardUrl = "https://kin.naver.com"
            boardList = [boardUrl+x for x in boardList]
            goBoard(boardList, pro_dict)
            curNum = curNum+1

In [247]:
def goBoard(boardList, pro_dict):
    for i in boardList:
        html = download("get", i, headers = headers)
        print(html.url)
        dom = BeautifulSoup(html.text, "lxml")
        try:
            content = dom.select_one("#content")
            question = content.select_one(".question-content .c-heading__title")
            if(question != None):
                question = question.text.strip()
            else:
                question = ""
            Qcontent = content.select_one(".c-heading__content")
            if(Qcontent!=None):
                Qcontent = Qcontent.text.strip()
            else:
                Qcontent = ""
            #print(question+Qcontent)
            answer = dom.select_one("#answerArea")
            answerlist = dom.select(".answer-content__list")
            for j in answerlist:
                if(pro_dict['name'] in j.select_one(".title a").text):
                    Acontent = j.select_one(".c-heading-answer__content-user").text.strip()
            board_info = {"boardUrl": html.url, "boardQC":question+Qcontent, "boardAC":Acontent}
            pushDb(board_info,1)
        except:
            print("비공개일겁니다.")

In [248]:
def pushDb(info, flag):
    if(flag == 1):
        cur.execute("""INSERT INTO board(boardUrl,boardQC, boardAC) VALUES('%s','%s','%s')"""
                %(info['boardUrl'], info['boardQC'], info['boardAC']
                  ))
        conn.commit()
    elif(flag == 0):
        cur.execute("""INSERT INTO doctor(docName,docLoc) VALUES('%s','%s')"""
                %(info['name'], info['loc']))
        conn.commit()